In [0]:
import numpy as np
from sklearn.linear_model import Ridge, ElasticNet, RANSACRegressor
import pandas as pd

Mount your Google Drive. In this notebook, we assume that 'report1' folder is placed directly under 'My Drive'.

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
!ls /content/drive/My\ Drive  #You should be able to see 'report1' folder by running this.

 00045.MTS
 00046.MTS
 20190611
 20190625
 20191220_mid_present.pdf
 answer20190514
'Colab Notebooks'
 day09
 DSC_0011.JPG
 handson20190514
 line_472323303084848.jpg
 line_472408253727160.jpg
 line_472430713069807.jpg
 myresearch.mp4
 myresearch_out.mp4
 report1
 sample.mp4
 sample_test01.mp4
 sample_test02.mp4
 se2019_list.pdf
'Unity 2018.3.14f1 Personal - test01.unity - bending_exp1 - PC, Mac & Linux Standalone _DX11_ 2019-12-06 21-14-16.mp4'


In [54]:
root_dir="/content/drive/My Drive/report1/"
N = 20000
nfeat = 8

train_data = pd.read_csv(root_dir+"train.csv")
test_data = pd.read_csv(root_dir+"test.csv")

Xmat = np.matrix(train_data)[:,:nfeat]
ymat = np.matrix(train_data)[:,nfeat]   #This is the target
print(train_data)
for i in range(len(train_data)):
  if int(Xmat[i,7][11:13]) <= 5:
    Xmat[i,7] = "am01"
  elif int(Xmat[i,7][11:13]) >= 6 and int(Xmat[i,7][11:13]) <= 11:
    Xmat[i,7] = "am02"
  elif int(Xmat[i,7][11:13]) >= 12 and int(Xmat[i,7][11:13]) <= 17:
    Xmat[i,7] = "pm01"
  elif int(Xmat[i,7][11:13]) >= 18 and int(Xmat[i,7][11:13]) <= 23:
    Xmat[i,7] = "pm02"
print(Xmat)

             holiday  temperature  ...            timestamp  traffic_volume
0      New Years Day       263.49  ...  2013-01-01 00:00:00            1439
1               None       263.78  ...  2013-01-01 01:00:00            1502
2               None       264.16  ...  2013-01-01 02:00:00             933
3               None       263.95  ...  2013-01-01 03:00:00             576
4               None       263.65  ...  2013-01-01 04:00:00             372
...              ...          ...  ...                  ...             ...
37691           None       249.84  ...  2017-12-31 19:00:00            2995
37692           None       249.79  ...  2017-12-31 20:00:00            2731
37693           None       249.61  ...  2017-12-31 21:00:00            2483
37694           None       249.50  ...  2017-12-31 22:00:00            2041
37695           None       249.45  ...  2017-12-31 23:00:00            1580

[37696 rows x 9 columns]
[['New Years Day' 263.49 0.0 ... 'Clouds' 'broken clouds' 'am0

Prepare the features.

In [55]:
#make dummy variables for the 'weather' categories
#note that we have to create dummies throughout training and testing data to align them
#(another idea is to use OneHotEncoder in scikit-learn)
all_data = pd.concat([train_data,test_data], sort=False)
print(all_data.columns.values)

for i in range(len(all_data)):
  #print(i)
  #print(all_data.iat[i,7])
  if int(all_data.iat[i,7][11:13]) <= 5:
    all_data.iat[i,7] = "am01"
  elif int(all_data.iat[i,7][11:13]) >= 6 and int(all_data.iat[i,7][11:13]) <= 11:
    all_data.iat[i,7] = "am02"
  elif int(all_data.iat[i,7][11:13]) >= 12 and int(all_data.iat[i,7][11:13]) <= 17:
    all_data.iat[i,7] = "pm01"
  elif int(all_data.iat[i,7][11:13]) >= 18 and int(all_data.iat[i,7][11:13]) <= 23:
    all_data.iat[i,7] = "pm02"
meta = pd.get_dummies(all_data['weather'])
meta2 = pd.get_dummies(all_data['timestamp'])

print(meta)
#X_tmp = np.float64(np.hstack([Xmat[:,4:5],meta.iloc[:len(train_data),:]])) #use four real-valued features and the dummy, and cast to float64 data type 
X_tmp = np.float64(np.hstack([np.hstack([Xmat[:,1:5], meta.iloc[:len(train_data),:]]), meta2.iloc[:len(train_data),:]]))
#X_tmp = np.float64(np.hstack([Xmat[:,1:5], meta.iloc[:len(train_data),:]]))
X = np.delete(X_tmp, [1,2], 1)
print(X)
y = np.float64(ymat)

XTrain = X[:N,:] #use the first N samples for training
yTrain = y[:N,:]
XVal = X[N:,:] #use the rests for validation
yVal = y[N:,:]

['holiday' 'temperature' 'rain_in_hour' 'snow_in_hour' 'clouds_cover'
 'weather' 'weather_detail' 'timestamp' 'traffic_volume']
      Clear  Clouds  Drizzle  Fog  ...  Smoke  Snow  Squall  Thunderstorm
0         0       1        0    0  ...      0     0       0             0
1         0       1        0    0  ...      0     0       0             0
2         0       0        0    0  ...      0     1       0             0
3         0       1        0    0  ...      0     0       0             0
4         0       1        0    0  ...      0     0       0             0
...     ...     ...      ...  ...  ...    ...   ...     ...           ...
7944      0       1        0    0  ...      0     0       0             0
7945      0       1        0    0  ...      0     0       0             0
7946      0       0        0    0  ...      0     0       0             1
7947      0       1        0    0  ...      0     0       0             0
7948      0       1        0    0  ...      0     0       

Run Ridge regression.

In [0]:
reg_elastic = Ridge(alpha=159.0).fit(XTrain, yTrain)   #alpha: regularization strength
#reg_elastic = ElasticNet(alpha=10000.0, l1_ratio=0.54).fit(XTrain, yTrain)
#reg_elastic.fit(XTrain, yTrain)
#reg_ransac = RANSACRegressor(reg_elastic, max_trials=100, min_samples=100, loss="absolute_loss", residual_threshold=4.0, random_state=0)
#reg_ransac.fit(XTrain, yTrain)

Evaluate training and validation errors.

In [80]:
def RMSE(vec1, vec2):
    return np.sqrt(np.mean(np.power(vec1-vec2,2)))

tmp_old =10000.0
tmp_alpha = 0

yHatTrain = reg_elastic.predict(XTrain)
yHatVal = reg_elastic.predict(XVal)

print("Training error ", RMSE(yTrain, yHatTrain))
print("Validation error ", RMSE(yVal, yHatVal))
'''
for i in range(20):
  reg_elastic = Ridge(alpha=10.0*(i+1)).fit(XTrain, yTrain)
  yHatTrain = reg_elastic.predict(XTrain)
  yHatVal = reg_elastic.predict(XVal)
  
  tmp = RMSE(yVal, yHatVal)
  print(i)
  print("Training error ", RMSE(yTrain, yHatTrain))
  print("Validation error ", RMSE(yVal, yHatVal))
  if tmp <= tmp_old:
    tmp_alpha = 10.0 * (i+1)
  tmp_old = tmp
print("best_alpha", tmp_alpha)
'''

Training error  1099.8782337146565
Validation error  1079.3965491111053


'\nfor i in range(20):\n  reg_elastic = Ridge(alpha=10.0*(i+1)).fit(XTrain, yTrain)\n  yHatTrain = reg_elastic.predict(XTrain)\n  yHatVal = reg_elastic.predict(XVal)\n  \n  tmp = RMSE(yVal, yHatVal)\n  print(i)\n  print("Training error ", RMSE(yTrain, yHatTrain))\n  print("Validation error ", RMSE(yVal, yHatVal))\n  if tmp <= tmp_old:\n    tmp_alpha = 10.0 * (i+1)\n  tmp_old = tmp\nprint("best_alpha", tmp_alpha)\n'

Now, project the testing data. You can upload 'result.txt' to the evaluation server.

In [0]:
#XTest_tmp = np.float64(np.hstack([test_data.iloc[:,4:5],meta.iloc[len(train_data):,:]]))
XTest = np.float64(np.hstack([test_data.iloc[:,1:5], meta.iloc[len(train_data):,:]]))
yHatTest = reg_elastic.predict(XTest)
np.savetxt(root_dir+'result.txt', yHatTest.T) #save predictions in rows